In [ ]:
# imports

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [ ]:
# I explored Foursquare website. I have registered and obtained API KEY. I created Environment Variable in my System, 
# to avoid using actual KEY in my function. I have found that in Foursquare website you can set parameters, choose language and get
# code. There are 2 different options https://location.foursquare.com/developer/reference/place-search & 
# https://location.foursquare.com/developer/recipes/place-search-using-latlong. I chose 2nd option (shown below).
# I decided to check Bars near one of the Bike stations. I set radius as 1000 meters. 

import requests
import os
api_key = os.environ["FOURSQUARE_API_KEY"]
url = "https://api.foursquare.com/v3/places/search"

params = {
  	"query": "bar",
  	"ll": "43.2391,76.9492", 
    "radius": "1000"
  	}

headers = {
    "Accept": "application/json",
    "Authorization": os.environ["FOURSQUARE_API_KEY"]
}

response = requests.request("GET", url, params=params, headers=headers)

print(response.text)

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
# The result of the Above request shows all the details of bars in that location

Put your parsed results into a DataFrame

In [ ]:
# The below function includes DataFrame

def get_foursquare_data():
    url = "https://api.foursquare.com/v3/places/search"

    params = {
        "query": "bar",
        "ll": "43.2391,76.9492", 
        "radius": "1000"
    }

    headers = {
        "Accept": "application/json",
        "Authorization": os.environ["FOURSQUARE_API_KEY"]
    }

    response = requests.get(url, params=params, headers=headers)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Failed to fetch Foursquare data. Status code: {response.status_code}")
        return None
    # then we need to make sure that "Response" giving us [200], if not, then "status_code" or ERROR
    
def dataframe_foursquare_data(foursquare_data):
    if foursquare_data is None:
        return None

    businesses = foursquare_data.get("results", [])

    if not businesses:
        print("No businesses found.")
        return None
    
    extracted_data = []
    for business in businesses: # Here we can choose all the information to be displayed
        business_data = {
            "Name": business.get("name", ""),
            "Categories": ", ".join(category.get("name", "") for category in business.get("categories", [])),
            "Distance": business.get("distance", ""),
            "Latitude": business.get("geocodes", {}).get("main", {}).get("latitude", ""),
            "Longitude": business.get("geocodes", {}).get("main", {}).get("longitude", ""),
            "Address": business.get("location", {}).get("formatted_address", ""),
            "Region": business.get("location", {}).get("region", ""),
            "Country": business.get("location", {}).get("country", ""),
            "Cross_Street": business.get("location", {}).get("cross_street", ""),
            "Link": business.get("link", ""),
            "Timezone": business.get("timezone", "")
        }
        extracted_data.append(business_data)

    # Convert the extracted data to a DataFrame
    df = pd.DataFrame.from_dict(extracted_data)
    return df

if __name__ == "__main__":
    foursquare_data = get_foursquare_data()
    if foursquare_data is not None:
        df = dataframe_foursquare_data(foursquare_data)
        print(df)

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [ ]:
# The same approach I used in YELP website. I have registered and obtained API KEY. I created Environment Variable in my System, 
# to avoid using actual KEY in my function.

import requests
import os
api_key = os.environ["YELP_API_KEY"]
url = "https://api.yelp.com/v3/businesses/search?latitude=43.2391&longitude=76.9492&term=bar&radius=1000"

headers = {
    "accept": "application/json",
    "Authorization": os.environ["YELP_API_KEY"]
    

response = requests.get(url, headers=headers)

print(response.text)

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
# Unfortunately, the Yelp Api has no information the location I chose and checked with Foursquare API. 

Put your parsed results into a DataFrame

In [ ]:
# Even the Yelp API did not provide me any information for the chosen location, I wrote the below function with DataFrame, 
# so it can be used for another location, where Yelp API has information. 

import requests
import pandas as pd
import os

def get_yelp_data():
    url = "https://api.yelp.com/v3/businesses/search"
    
    # Replace "<YOUR_YELP_ACCESS_TOKEN>" with your actual Yelp access token
    headers = {
        "accept": "application/json",
        "Authorization": os.environ["YELP_API_KEY"]
    }
    
    params = {
        "latitude": 43.2391,
        "longitude": 76.9492,
        "term": "bar",
        "radius": 1000
    }

    response = requests.get(url, params=params, headers=headers)

    if response.status_code == 200:
        data = response.json()
          
        return data
    else:
        print(f"Failed to fetch Yelp data. Status code: {response.status_code}")
        return None

def create_dataframe_from_yelp_data(yelp_data):
    if yelp_data is None:
        return None

    bars = yelp_data.get("bar", [])
    
    if not bars:
        print("No bars found.")
        return None

    # Extract the relevant information from each business
    extracted_data = []
    for bar in bars:
        bars_data = {
            "Name": bars.get("name", ""),
            "Rating": bars.get("rating", ""),
            "Price": bars.get("price", ""),
            "Location": bars.get("location", {}).get("address1", ""),
            "Phone": bars.get("phone", "")
        }
        extracted_data.append(bar_data)

    # Convert the extracted data to a DataFrame
    df = pd.DataFrame.from_dict(extracted_data)
    return df

if __name__ == "__main__":
    yelp_data = get_yelp_data()
    if yelp_data is not None:
        df = create_dataframe_from_yelp_data(yelp_data)
        print(df)

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

<!-- As I already mentioned earlier, the Yelp API did not find any POIs in Almaty. 
Foursquare API, on other hand, found a big number of bars and restaurants in Almaty and at the exact location, when I provided latitude/longitude. I could get information like "Name", "Categories", "Address" and even I could give limitation of radius = 1000 meters. -->


Get the top 10 restaurants according to their rating

In [ ]:
# Since Yelp did not provide any results to me, I checked ratings in Foursquare and found that Rating information is missing.

def get_foursquare_data():
    url = "https://api.foursquare.com/v3/places/search"

    params = {
        "query": "bar",
        "ll": "43.2391,76.9492", 
        "radius": "1000"
    }

    headers = {
        "Accept": "application/json",
        "Authorization": os.environ["FOURSQUARE_API_KEY"]
    }

    response = requests.get(url, params=params, headers=headers)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Failed to fetch Foursquare data. Status code: {response.status_code}")
        return None
    
def dataframe_foursquare_data(foursquare_data):
    if foursquare_data is None:
        return None

    businesses = foursquare_data.get("results", [])

    if not businesses:
        print("No businesses found.")
        return None
    
    extracted_data = []
    for business in businesses:
        rating_data = business.get("rating", None)
        business_data = {
            "Name": business.get("name", ""),
            "Categories": ", ".join(category.get("name", "") for category in business.get("categories", [])),
            "Rating": rating_data if rating_data is not None else "N/A",  # Adding the rating or "N/A" if not available
        }
        extracted_data.append(business_data)

    # Convert the extracted data to a DataFrame
    df = pd.DataFrame.from_dict(extracted_data)
    return df

if __name__ == "__main__":
    foursquare_data = get_foursquare_data()
    if foursquare_data is not None:
        df = dataframe_foursquare_data(foursquare_data)
        print(df)